In [ ]:
import numpy as np
import copy
import random as rd
import cProfile
import pstats
import timeit
SIZE = 5


In [ ]:
i = [(1,2),(2,3),(3,4)]
_dic = {}
for j in i:
  _dic[j] = True

_dic.get(i[0])

True

In [ ]:
b = np.matrix('''
1 0 -1 1 1;
-1 1 0 0 1;
1 0 1 1 -1;
0 0 0 1 0;
0 0 0 0 -1
''')
bl = (np.logical_not(b.ravel()).tolist()[0])
#wh = list((b.ravel() == 1))[0]
#bll = (b.ravel() == 0).tolist()[0]
#wh = list(b1.flatten() == 1)
#bl = b.flatten() == 0
#wh = b.flatten() == 1
print(bl)
#print(bll)
#print(wh)
#print(bl==wh)
#print(store)
se = """import numpy as np
b = np.matrix('''
1 0 -1 1 1;
-1 1 0 0 1;
1 0 1 1 -1;
0 0 0 1 0;
0 0 0 0 -1
''')
bl = (b.flatten() == 0).tolist()[0]
wh = (b.flatten() == 1).tolist()[0]
"""
print(b.ravel() == 0)
print(timeit.timeit(setup = se,
                    stmt = 'np.logical_not(b.ravel()).tolist()[0]',
                    number = 10000))

[False, True, False, False, False, False, False, True, True, False, False, True, False, False, False, True, True, True, False, True, True, True, True, True, False]
[[False  True False False False False False  True  True False False  True
  False False False  True  True  True False  True  True  True  True  True
  False]]
0.0679050760000166


In [ ]:
print(val_matrix(game3.board.tolist(), 1, []))
game3.display()

[(1, 1), (0, 2), (0, 0)]
{(1, 1): True} True
{(0, 2): True} True
{(0, 0): True} False
[(0, 2)]
{(0, 2): True} True
[(0, 2), (1, 1)]
{(0, 2): True} True
{(1, 1): True} True
[]
[(1, 1)]
{(1, 1): True} True
[]
[]
[]
[]
[]
[]
([[-inf, 1, -inf, 0], [-inf, -inf, 0, 0], [-inf, 0, 0, 0], [0, 0, 0, 0]], [[False, [[-1, 1, 0, -1], [1, 0, -1, -1], [1, -1, -1, -1], [-1, -1, -1, -1]], False, [[0, -1, 0, 1], [1, 0, -1, -1], [1, -1, -1, -1], [-1, -1, -1, -1]]], [False, False, [[0, -1, 0, -1], [1, 0, 1, -1], [1, -1, -1, -1], [-1, -1, -1, -1]], [[0, -1, 0, -1], [1, 0, -1, 1], [1, -1, -1, -1], [-1, -1, -1, -1]]], [False, [[0, -1, 0, -1], [1, 0, -1, -1], [1, 1, -1, -1], [-1, -1, -1, -1]], [[0, -1, 0, -1], [1, 0, -1, -1], [1, -1, 1, -1], [-1, -1, -1, -1]], [[0, -1, 0, -1], [1, 0, -1, -1], [1, -1, -1, 1], [-1, -1, -1, -1]]], [[[0, -1, 0, -1], [1, 0, -1, -1], [1, -1, -1, -1], [1, -1, -1, -1]], [[0, -1, 0, -1], [1, 0, -1, -1], [1, -1, -1, -1], [-1, 1, -1, -1]], [[0, -1, 0, -1], [1, 0, -1, -1], [1, -1, -1, -1]

In [ ]:
def cluster(board, tup, liberty=False):      # Find the cluster starting from x,y recursively and determine whether this group is alive or dead

  SIZE = len(board)
  x, y = tup
  target = board[x][y]
  toDo = [[x,y]]
  record = {tup : True}
  #record[tup] = True

  while toDo:
    nx = toDo[0][0]
    ny = toDo[0][1]
    toTry = [(nx-1,ny), (nx+1,ny), (nx,ny-1), (nx,ny+1)]
    for i in toTry:
      if i[0] >= 0 and i[0] < SIZE and i[1] >= 0 and i[1] < SIZE:
        if board[i[0]][i[1]] == target:
          if not record.get(i):
            toDo.append(i)
            record[i] = True
        elif not liberty and board[i[0]][i[1]] == -1:
          liberty = True

    toDo = toDo[1:]
  return record, liberty


def outcome(board, x, y):      # Determine the impact of a move placed at x,y in terms of immediate capture, and the subsequent board after deleting dead clusters

  result = 0
  check = not board[x][y]
  toVisit = []
  suicide = True
  SIZE = len(board)
                
  for i in [(x+1,y), (x-1,y), (x,y+1), (x,y-1)]:
    if i[0] >= 0 and i[0] < SIZE and i[1] >= 0 and i[1] < SIZE:
      if board[i[0]][i[1]] == check:
        toVisit.append(i)     
      elif board[i[0]][i[1]] == -1 and suicide:
        suicide = False

  #print(toVisit)
  visited = {}
  dead = {}
  for i in toVisit:
    if not visited.get(i):
      group, alive = cluster(board, i)
      visited.update(group)
      if not alive:
        result += len(group)
        dead.update(group)
  
  if not result and suicide:
    #if not visited.get((x, y)):
    group, alive = cluster(board, (x, y))
    if not alive:
      result = -len(group)
      dead = group
  
  for i in dead:
    board[i[0]][i[1]] = -1

  return result, board


def checkHistory(board, history):     #check if the position has appeared in the last 5 steps
  cBoard = compressor(board)
  for i in history[-5:]:
    if cBoard[0] == i[0] and cBoard[1] == i[1]:
      return True
  return False


def val_matrix(board, turn, history):    #return a matrix of playable locations and their instant value

  n = len(board)
  result = [[float('-inf')] * n for i in range(n)]
  outBoard = []

  for x in range(n):
    Row = [False] * n
    for y in range(n):
      tmp = [[i  for i in j] for j in board]
      if tmp[x][y] == -1:
        tmp[x][y] = turn
        result[x][y], tmp = outcome(tmp, x, y)    # determine the instant value if placing at x, y

        if checkHistory(tmp, history):
          result[x][y] = -1
        else:
          Row[y] = tmp
    outBoard.append(Row)
      
  return result, outBoard


def compressor(board):         #  compress a given board into two flattened bool list, 1st black, 2nd white
  n = len(board)
  black = [False] * (n * n)
  white = [False] * (n * n)
  for i in range(n):
    for j in range(n):
      if not board[i][j]:
        black[i * n + j] = True
      elif board[i][j] == 1:
        white[i * n + j] = True
  #black = [i for j in black for i in j]
  #white = [i for j in white for i in j]
  #b = np.array(board)
  #black = np.logical_not(b.ravel()).tolist()
  #black = (b.ravel() == 0).tolist()
  #white = (b.ravel() == 1).tolist()
  return [black, white]

In [ ]:
class Game:

  history = []
  board = None
  turn = None
  size = None
  score = []

  def __init__(self, n):
    self.board = np.full((n,n), -1)
    self.turn = 0
    self.size = n
    self.score = [0,0]
    self.history = []

  def init_with(self, n, t, b, score):      # for Monte_Carlo play, board is always as python list instead of np array
    self.board = b                            # history is ignored
    self.size = n
    self.turn = t
    self.score = [0,0]
    self.history = []
    self.score = score
    cand = [(i,j) for j in range(SIZE) for i in range(SIZE) if self.board[i][j] == -1]
    r = rd.randrange(len(cand))
    return cand[r]

  def play(self, x, y):                       # proceed to play at x,y and record score, then return max score

    self.history.append(self.board)
    newstate = self.board.tolist()
    newstate[x][y] = self.turn
    pts, board = outcome(newstate, x, y)
    self.board = np.array(board)
    self.score[self.turn] += pts
    self.turn = not self.turn

    return max(self.score)
  
  def MC_play(self, x, y):                       # quick version of play, without recording history to save time and space
                                                # Check of empty place is assumed to be done beforehand
    newstate = [[i for i in j] for j in self.board]
    newstate[x][y] = self.turn
    pts, board = outcome(newstate, x, y)

    cand = [(i,j) for j in range(SIZE) for i in range(SIZE) if board[i][j] == -1]
    if not len(cand) or (pts < 0 and len(cand) == 1):     # If no space or only move left is suicidal
      return False                                 # return False for suicide check
    self.score[self.turn] += pts
    self.board = board
    self.turn = not self.turn
    r = rd.randrange(len(cand))                   # random move
    return cand[r]

  def display(self, b = []):
    if not len(b):
      b = self.board
    print(f'\nScore: B/{self.score[0]}  W/{self.score[1]}')
    for row in b:
      for each in row:
        if not each:
          print('x', end=' ')
        elif each > 0:
          print('o', end=' ')
        else:
          print('-', end=' ')
      print()
    print()
  
  def replay(self, hist = None):
    if not hist:
      hist = self.history
    for i,j in enumerate(hist):
      print(f'{i}:')
      self.display(j)

  def winner(self):
    if self.score[0] == self.score[1]:
      return 'D'
    elif self.score[0] > self.score[1]:
      return 'B'
    else:
      return 'W'

In [ ]:
class player:

  skill = ''
  pick = None
  name = ''

  def __init__(name, strat):
    self.name = name
    self.pick = strat
    self.skill = str(strat)

In [ ]:
game4 = Game(SIZE)
game4.play(1,1)
game4.play(2,2)
game4.play(0,2)
game4.play(1,2)
game4.play(2,1)
game4.play(3,1)

game4.play(3,2)
game4.play(4,2)
game4.play(4,1)
game4.play(3,4)
game4.play(3,0)

game4.display()


Score: B/1  W/0
- - x - - 
- x o - - 
- x o - - 
x - x - o 
- x o - - 



In [ ]:
def PromAndRandom(matrix, prob = 0.3):    # Generate the [True, False] matrix for the current position with the desired probability, default is 0.3
  n = len(matrix)
  result = [[False] * n for i in range(n)]
  for i in range(n):
    for j in range(n):
      if matrix[i][j] > -100:
        result[i][j] = True
      elif not matrix[i][j]:
        if rd.random() < prob:
          result[i][j] = True
  return result

def DoItAll(matrix):
  n = len(matrix)
  result = [[False] * n for i in range(n)]
  for i in range(n):
    for j in range(n):
      if matrix[i][j] > -100:
        result[i][j] = True
  return result

def filterExc(matrix, v):
  ans = matrix.copy()
  ans[ans != v] = -1
  return ans


In [ ]:
class node:

  board = None  
  matrix = None        # Instant value matrix
  path = None         # Store of board positions one move later
  children = None     # Direct children list of this node
  parent = None       # Direct parent
  value = None         # Value obtained from moving here
  eV = None           # expected value, calculated from instant value and sum of all children recursively
  turn = None         # which turn is that in this node
  history = None         # past 5 board history, compressed to list[black[], white[]]

  def __init__(self, board, turn, history, val = 0):
    n = len(board)
    self.history = history
    self.board = board.copy()
    self.children = []
    self.value = val
    self.turn = turn
    self.parent = None
    self.eV = None

    self.matrix, self.path = val_matrix(board, turn, history)
    self.last_location = self.last()

  def find(self, board):
    for i in self.children:
      if i.board == board:
        return i
    return False
  

  def last(self):                 #find last step by comparing with history
    last = self.history[-1]
    n = len(self.board)
    if len(last):
      thisCom = compressor(self.board)
      for i in range(n * n):
        if last[not self.turn][i] ^ thisCom[not self.turn][i]:
          return i // n, i % n
    else:
      print("history not found")
      return


  def proceed(self, x ,y):         # construct node if we play at x,y
      n_hist = self.history[-4:]      # save board state to history
      n_hist.append(compressor(self.board))
      child = node(self.path[x][y], not self.turn, n_hist, self.matrix[x][y])
      child.parent = self
      self.children.append(child)
      self.matrix[x][y] = float('-inf')     #  Mark this place as already tried
      return child

    #else:
      #print(f"Move invalid at {x}, {y}")
      #return False
  
  def expand(self):                       # Create a node for every possible move on the board
    doMatrix = DoItAll(self.matrix)
    n = len(self.board)
    for x in range(n):
      for y in range(n):
        if self.matrix[x][y] >= 0 and doMatrix[x][y]:
          self.proceed(x,y)
    return self.children

  def selective(self):                    # only proceed on those who have a promising result, randomly on others
    doMatrix = PromAndRandom(self.matrix)
    n = len(self.board)
    for x in range(n):
      for y in range(n):
        if doMatrix[x][y] and self.matrix[x][y] >= 0:
          self.proceed(x,y)
    return self.children

  def update(self):                 # calculate expected value by looking into values of children
    n = len(self.children)
    if not n:
      flat = [i for j in self.matrix for i in j]
      self.eV = self.value - 0.8 * max(flat)
    else:
      tmp = [i.eV for i in self.children]
      if len(tmp):                                  # the value of the next level is a gain to opponent, hence a loss
        self.eV = self.value - 0.8 * max(tmp)       # 0.8 is the deflation factor of each step
      else:
        self.eV = self.value
      

class GameTree:   # Upper hierachy of the nodes

  state = None
  head = None
  P1 = None
  P2 = None

  def __init__(self, game, strat1, strat2):

    cHistory = [compressor(i) for i in game.history]
    self.state = game
    self.head = node(game.board.tolist(), game.turn, cHistory)
    self.P1 = strat1
    self.P2 = strat2

  def show(self, board):
    for i in range(len(board)):
      for j in range(len(board)):
        if board[i][j] < 0:
          print('.', end = '')
        elif not board[i,j]:
          print('x', end = '')
        else:
          print('o', end = '')
      print()
    print()

  def move(self, x,y):                          # Game is proceeded by playing at x,y
    m = self.state.play(x,y)
    
    if m >= 10:
      return True                               # return true if game over

    for i in self.head.children:
      if i.last() == (x,y):
        self.head = i
        return False
        # print("found node, proceeding")
    print("ERROR: not found")
    return False

  def compete(self, display = True):
    end = False
    i = 0
    while not end:
      if not i%2:
        m = self.P1(self)                       # P1 plays first
      else:
        m = self.P2(self)                       # P2 plays next
      i += 1
      if not m:
        end = True
      elif self.move(m[0], m[1]):
        end = True
    if display:
      print(f'End: {i} steps')
    return self.state.score

  def lookForward(self, n, r=0):          # n = full expansion, r is selective
    trace = []
    digQueue = []
    nextList = [self.head]

    for i in range(n + r):
      digQueue = nextList
      nextList = []
      if i < n:
        for each in digQueue:
          nextList.extend(each.expand())
      else:
        for each in digQueue:
          nextList.extend(each.selective())
      trace = nextList + trace
      # print(len(trace))
    
    for i in trace:
      i.update()
    
    self.head.children.sort(key = lambda x: x.eV, reverse=True)
    
  def digDeep(self, num, n, r):           # Dig much deeper into the top (NUM) choices, full expanding by n steps, then randomly by r steps
    choices = self.head.children
    choices.sort(key = lambda x: x.eV, reverse=True)
    queue = choices[:num]
    trace = choices[:num]
    toDo = []

    for i in range(n + r):
      print(len(queue))
      if i < n:
        for each in queue:
          toDo.extend(each.expand())        # Expand this node fully
      else:
        for each in queue:
          toDo.extend(each.selective())     # Expand this node randomly

      trace = toDo + trace
      queue = toDo
      toDo = []
      #print(len(trace))

    for i in trace:
      i.update()
      
    #tmp = [i.eV for i in choices]
    #print(tmp)

  def displayTree(self, depth, optimal = False):
    queue = [self.head]
    vals = [self.head.value]
    count = 0
    while len(queue) and count < depth:
      n = len(queue)
      size = len(self.head.board)
      for v in vals:
        if v < 0:
          print(f' {v:.1f}  ', end='')
        else:
          print(f' +{v:.1f}  ', end='')
      print()
      vals = []
      for i in range(size):
        for j in range(n):
          for k in range(size):
            if queue[j].board[i][k] < 0:
              print('.', end = '')
            elif not queue[j].board[i][k]:
              print('x', end = '')
            else:
              print('o', end = '')
          print('  ', end='')
        print()
      print()
      count += 1

      if not optimal:
        for i in range(n):
          for j in queue[i].children:
            #if j.eV:           #  omit 0 values
            queue.append(j)
            vals.append(j.eV)
      else:
        tmp = []
        maxi = 0
        for i in queue:
          if len(i.children):
            tmp.append(max([j.eV for j in i.children]))
        if len(tmp):
          maxi = max(tmp)
        for i in queue:
          for j in i.children:
            if j.eV >= maxi:
              queue.append(j)
              vals.append(j.eV)

      queue = queue[n:]



In [ ]:
# strategies

def beginner(tree):                                                   # choose from all legal moves at random
  tree.lookForward(1)
  choices = [i.last_location for i in tree.head.children]
  if not choices:
    return False                                                      # return False for no possible moves
  mv = rd.randrange(len(choices))
  return choices[mv]

def forwardLooker(tree):                                              # look 2 step ahead of time
  tree.lookForward(2)
  choices = tree.head.children
  if not choices:
    return False
  best = max(choices, key=lambda i: i.eV)                             # make a list of all best moves
  final = [i for i in choices if i.eV == best.eV]

  mv = rd.randrange(len(final))
  return final[mv].last_location

def FLook_2_2(tree):                                              # look 2 step ahead of time, followed by 2 steps in random accuracy
  tree.lookForward(2,1)
  choices = tree.head.children
  if not choices:
    return False
  best = max(choices, key=lambda i: i.eV)                             # make a list of all best moves
  final = [i for i in choices if i.eV == best.eV]

  mv = rd.randrange(len(final))
  return final[mv].last_location


def MonteCarlo(tree):

  tree.lookForward(1)
  choices = tree.head.children
  if not choices:
    return False
  store = []
  role = tree.head.turn                                   # play as yourself

  for i in choices:
    res = [0,1]                                               # total games start with 1 to prevent division by 0 afterwards

    for e in range(100):
      g = Game(SIZE)
      move = g.init_with(SIZE, not role, i.board, tree.state.score.copy())
      step = 0

      while step < 100:                                       # at  most 100 steps allowed
        tmp = g.MC_play(move[0], move[1])
        if max(g.score) >= 10:
          if g.score[role] > g.score[not role]:               # win or not depends on the role you are using
            res[0] += 1
          else:
            res[1] += 1
          break
        if not tmp:
          break
        move = tmp
        step += 1
      
    store.append(res[0] / sum(res))                    

  # Evaluate and return

  a = max(store)
  for i in range(len(store)):
    if store[i] == a:
      return choices[i].last_location

  return store


In [ ]:
#tree.head.turn
MonteCarlo(tree)

[0.5561224489795918,
 0.5117493472584856,
 0.5803108808290155,
 0.6041131105398457,
 0.5687830687830688,
 0.5654450261780105,
 0.524547803617571,
 0.5438144329896907,
 0.5673575129533679,
 0.6198979591836735,
 0.5345268542199488,
 0.6134020618556701,
 0.5552631578947368,
 0.5419847328244275,
 0.5235602094240838,
 0.5153846153846153,
 0.515625]

In [ ]:
tree.lookForward(2,1)
tree.displayTree(2)

 +0.0  
.ox..  
.x...  
o....  
x.x..  
oo...  

 +0.8   +0.2   -0.2   -0.2   -0.2   -0.2   -0.2   -0.2   -0.2   -0.2   -0.2   -0.2   -0.2   -0.2   -0.2   -0.2   -0.2  
.ox..  x.x..  .oxx.  .ox.x  .ox..  .ox..  .ox..  .ox..  .ox..  .ox..  .ox..  .ox..  .ox..  .ox..  .ox..  .ox..  .ox..  
.x...  .x...  .x...  .x...  xx...  .xx..  .x.x.  .x..x  .x...  .x...  .x...  .x...  .x...  .x...  .x...  .x...  .x...  
o....  o....  o....  o....  o....  o....  o....  o....  ox...  o.x..  o..x.  o...x  o....  o....  o....  o....  o....  
xxx..  x.x..  x.x..  x.x..  x.x..  x.x..  x.x..  x.x..  x.x..  x.x..  x.x..  x.x..  x.xx.  x.x.x  x.x..  x.x..  x.x..  
oo...  oo...  oo...  oo...  oo...  oo...  oo...  oo...  oo...  oo...  oo...  oo...  oo...  oo...  oox..  oo.x.  oo..x  



In [ ]:
def normal_game():
  game = Game(SIZE)
  game.play(1,1)
  game.play(3,3)
  return game

def playForRounds(p1, p2, n, show = False):
  result = [0,0,0]        # Black win, White win, Draw

  for i in range(n):
    print(f'Game {i}')
    game1 = normal_game()
    tree = GameTree(game1, p1, p2)
    #tree.lookForward(1,0)

    tree.compete(False)

    if game1.winner() == 'B':
      result[0] += 1
    elif game1.winner() == 'W':
      result[1] += 1
    else:
      result[2] += 1
    
    if show:
      game1.display()
    
  return result

In [ ]:
#del game3
game3 = Game(SIZE)
game3.play(1,1)
game3.play(0,1)
game3.play(0,2)
game3.play(2,0)
game3.play(3,0)
game3.play(4,0)
game3.play(3,2)
game3.play(4,1)
#game3.play(4,3)
#game3.play(2,3)
game3.display()
tree = GameTree(game3, beginner, forwardLooker)
#game3.replay()
#tree.show(tree.state.board)


Score: B/0  W/0
- o x - - 
- x - - - 
o - - - - 
x - x - - 
o o - - - 



In [ ]:
playForRounds(beginner, MonteCarlo, 10, False)
#game1 = normal_game()
#tree = GameTree(game1, MonteCarlo, forwardLooker)
#tree.lookForward(1,0)

#tree.compete()

Game 0
Game 1
Game 2
Game 3
Game 4
Game 5
Game 6
Game 7
Game 8
Game 9


[4, 4, 2]

In [ ]:
#tree.state.score
game1.replay()

In [ ]:
#prof = cProfile.Profile()
tree = GameTree(game3, beginner, forwardLooker)
#prof.enable()

tree.lookForward(1,0)

#prof.disable()
#tree.digDeep(3, 2, 2)
#tree.show(game1.board)
#print(len(game1.replay()))

In [ ]:
tree.compete(False)

[12, 0]

In [ ]:
tree.displayTree(2)

 +0.0  
x.xx.  
xxx.x  
oooxx  
.ooox  
xooox  

 +9.0   +5.6   +5.6   +5.6  
x.xx.  xxxx.  x.xxx  x.xx.  
xxx.x  xxx.x  xxx.x  xxxxx  
...xx  oooxx  oooxx  oooxx  
x...x  .ooox  .ooox  .ooox  
x...x  xooox  xooox  xooox  



In [ ]:
game3.winner()

'B'

In [ ]:
li = []
if li:
  print(True)
else:
  print(False)

False


In [ ]:
forwardLooker(tree)

21
440


(0, 0)

In [ ]:
tree.move(2,2)

TypeError: ignored

In [ ]:
tree.displayTree(2)

 +0.0  
.ox..  
.x...  
o....  
.....  
.....  

 +1.0   +0.6   +0.6   +0.6   +0.6   +0.6   +0.6   +0.6   +0.6   +0.6   +0.6   +0.6   +0.6   +0.6   +0.6   +0.6   +0.6   +0.6   +0.6   +0.6   +0.6  
x.x..  .oxx.  .ox.x  .ox..  .ox..  .ox..  .ox..  .ox..  .ox..  .ox..  .ox..  .ox..  .ox..  .ox..  .ox..  .ox..  .ox..  .ox..  .ox..  .ox..  .ox..  
.x...  .x...  .x...  xx...  .xx..  .x.x.  .x..x  .x...  .x...  .x...  .x...  .x...  .x...  .x...  .x...  .x...  .x...  .x...  .x...  .x...  .x...  
o....  o....  o....  o....  o....  o....  o....  ox...  o.x..  o..x.  o...x  o....  o....  o....  o....  o....  o....  o....  o....  o....  o....  
.....  .....  .....  .....  .....  .....  .....  .....  .....  .....  .....  x....  .x...  ..x..  ...x.  ....x  .....  .....  .....  .....  .....  
.....  .....  .....  .....  .....  .....  .....  .....  .....  .....  .....  .....  .....  .....  .....  .....  x....  .x...  ..x..  ...x.  ....x  



In [ ]:
tree.displayTree(2)

 +0.0  
.ox..  
.x...  
o....  
.....  
.....  

 +1.0   +0.0   +0.0   +0.0   +0.0   +0.0   +0.0   +0.0   +0.0   +0.0   +0.0   +0.0   +0.0   +0.0   +0.0   +0.0   +0.0   +0.0   +0.0   +0.0   +0.0  
x.x..  .oxx.  .ox.x  .ox..  .ox..  .ox..  .ox..  .ox..  .ox..  .ox..  .ox..  .ox..  .ox..  .ox..  .ox..  .ox..  .ox..  .ox..  .ox..  .ox..  .ox..  
.x...  .x...  .x...  xx...  .xx..  .x.x.  .x..x  .x...  .x...  .x...  .x...  .x...  .x...  .x...  .x...  .x...  .x...  .x...  .x...  .x...  .x...  
o....  o....  o....  o....  o....  o....  o....  ox...  o.x..  o..x.  o...x  o....  o....  o....  o....  o....  o....  o....  o....  o....  o....  
.....  .....  .....  .....  .....  .....  .....  .....  .....  .....  .....  x....  .x...  ..x..  ...x.  ....x  .....  .....  .....  .....  .....  
.....  .....  .....  .....  .....  .....  .....  .....  .....  .....  .....  .....  .....  .....  .....  .....  x....  .x...  ..x..  ...x.  ....x  



In [ ]:
tree.state.replay()

0:

Score: B/1  W/0
- - - - - 
- - - - - 
- - - - - 
- - - - - 
- - - - - 

1:

Score: B/1  W/0
- - - - - 
- x - - - 
- - - - - 
- - - - - 
- - - - - 

2:

Score: B/1  W/0
- o - - - 
- x - - - 
- - - - - 
- - - - - 
- - - - - 

3:

Score: B/1  W/0
- o x - - 
- x - - - 
- - - - - 
- - - - - 
- - - - - 

4:

Score: B/1  W/0
- o x - - 
- x - - - 
o - - - - 
- - - - - 
- - - - - 

5:

Score: B/1  W/0
x - x - - 
- x - - - 
o - - - - 
- - - - - 
- - - - - 



In [ ]:
ps = pstats.Stats(prof)
ps.print_stats()

         7663291 function calls (6732619 primitive calls) in 11.450 seconds

   Random listing order was used

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   389990    0.114    0.000    0.114    0.000 {method 'update' of 'dict' objects}
  3047667    0.263    0.000    0.263    0.000 {method 'append' of 'list' objects}
   175896    0.061    0.000    0.061    0.000 {method 'insert' of 'list' objects}
    11135    0.003    0.000    0.003    0.000 {method 'extend' of 'list' objects}
        1    0.000    0.000    0.000    0.000 {method 'sort' of 'list' objects}
    47826    0.018    0.000    0.018    0.000 {method 'split' of 'str' objects}
       12    0.000    0.000    0.000    0.000 {built-in method posix.getpid}
       17    0.000    0.000    0.000    0.000 {built-in method posix.urandom}
       17    0.000    0.000    0.000    0.000 {method 'acquire' of '_thread.lock' objects}
    64561    0.009    0.000    0.009    0.000 {method 'random' of '_random.Random' 

In [ ]:
ps = pstats.Stats(prof)
ps.print_stats()

         6387960 function calls in 9.027 seconds

   Random listing order was used

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
  1375722    0.213    0.000    0.213    0.000 {method 'get' of 'dict' objects}
   361136    0.106    0.000    0.106    0.000 {method 'update' of 'dict' objects}
  1127817    0.153    0.000    0.153    0.000 {method 'append' of 'list' objects}
    11114    0.003    0.000    0.003    0.000 {method 'extend' of 'list' objects}
        1    0.000    0.000    0.000    0.000 {method 'sort' of 'list' objects}
       12    0.000    0.000    0.000    0.000 {built-in method posix.getpid}
       17    0.000    0.000    0.000    0.000 {built-in method posix.urandom}
       17    0.000    0.000    0.000    0.000 {method 'acquire' of '_thread.lock' objects}
    64701    0.008    0.000    0.008    0.000 {method 'random' of '_random.Random' objects}
        2    0.000    0.000    0.000    0.000 {built-in method builtins.compile}
        2    0.000  

In [ ]:
ps = pstats.Stats(prof)
ps.print_stats()

         8883051 function calls in 11.734 seconds

   Random listing order was used

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
  1768275    0.286    0.000    0.286    0.000 {method 'get' of 'dict' objects}
   504927    0.133    0.000    0.133    0.000 {method 'update' of 'dict' objects}
    63002    0.015    0.000    0.015    0.000 {method 'copy' of 'list' objects}
  1491382    0.194    0.000    0.194    0.000 {method 'append' of 'list' objects}
     8794    0.003    0.000    0.003    0.000 {method 'extend' of 'list' objects}
        1    0.000    0.000    0.000    0.000 {method 'sort' of 'list' objects}
       10    0.000    0.000    0.000    0.000 {built-in method posix.getpid}
       13    0.000    0.000    0.000    0.000 {method 'acquire' of '_thread.lock' objects}
       13    0.000    0.000    0.000    0.000 {method 'append' of 'collections.deque' objects}
        2    0.000    0.000    0.000    0.000 {built-in method builtins.compile}
        2    0

In [ ]:
#tree3 = GameTree(game3)
tree3.digDeep(2,5,2)


2
18
142
963
5868
29059
86025


In [ ]:
tree3.displayTree(3)

=1.0  
.ox.  
ox..  
ox..  
....  

=1.1  =1.1  =1.1  =1.1  =1.1  =1.1  =1.1  =0.9  =0.4  
.ox.  .ox.  .ox.  .ox.  .ox.  .ox.  .oxx  .ox.  .ox.  
ox..  oxx.  ox.x  ox..  ox..  ox..  ox..  ox..  ox..  
oxx.  ox..  ox..  ox.x  ox..  ox..  ox..  ox..  ox..  
....  ....  ....  ....  .x..  ..x.  ....  ...x  x...  

-3.4  -1.8  -1.8  -1.6  -1.7  -1.4  -2.3  -1.5  -1.4  -1.4  -3.2  -1.7  -1.6  -1.6  -1.6  -2.3  -1.4  -1.6  -1.5  -1.5  -3.2  -2.0  -1.4  -1.4  -2.3  -1.4  -1.4  -2.3  -3.2  -2.0  -1.5  -1.4  -1.4  -1.5  -2.0  -2.0  -1.4  -1.8  -3.2  -1.8  -1.8  -1.8  -3.2  -1.8  -1.5  -1.8  -3.2  -2.0  -1.8  -1.4  -2.3  -2.3  -2.0  -1.5  -3.2  -1.5  -1.3  -1.4  -1.5  -2.3  -1.5  -1.4  -1.4  -1.2  -1.5  -3.2  -1.6  -1.4  -1.4  -1.9  -1.2  -2.4  -2.4  -2.4  -0.5  -2.4  -2.4  -2.4  -2.4  
oox.  .oxo  .ox.  .ox.  .ox.  .ox.  .ox.  .ox.  .ox.  .ox.  oox.  .oxo  .ox.  .ox.  .ox.  .ox.  .ox.  .ox.  .ox.  .ox.  oox.  .ox.  .ox.  .ox.  .ox.  .ox.  .ox.  .ox.  oox.  .oxo  .ox.  .ox.  .ox.  .ox.  .ox.  .ox

In [ ]:
tree3.move(0,1)

found node, proceeding


True

In [ ]:
tree3.displayTree(2)

=0.0  
xxx.  
.x.o  
oxxo  
xxxo  

-4.2  -4.3  
xxxo  xxx.  
.x.o  .xoo  
oxxo  oxxo  
xxxo  xxxo  



In [ ]:
tree3.displayTree(3)

=1.0  
.ox.  
ox..  
ox..  
....  

=1.5  =1.4  =1.0  =1.0  =0.9  =0.9  =0.9  =0.9  =0.3  
.ox.  .ox.  .ox.  .ox.  .oxx  .ox.  .ox.  .ox.  .ox.  
ox..  ox..  ox..  ox.x  ox..  oxx.  ox..  ox..  ox..  
oxx.  ox..  ox..  ox..  ox..  ox..  ox.x  ox..  ox..  
....  .x..  ..x.  ....  ....  ....  ....  ...x  x...  

-3.7  -2.0  -2.1  -1.8  -2.0  -2.7  -1.8  -2.1  -2.0  -1.8  -1.8  -3.5  -1.8  -1.7  -1.8  -1.8  -3.5  -1.9  -1.5  -2.3  -3.2  -1.3  -1.3  -1.8  -1.4  -2.3  -1.4  -1.7  -1.4  -2.3  -1.4  -3.2  -1.2  -1.4  -1.4  -1.4  -1.2  -3.2  -1.3  -1.3  -1.4  -2.3  -1.2  -1.4  -1.7  -1.4  -1.4  -3.2  -1.4  -1.2  -2.3  -1.5  -1.4  -1.2  -1.4  -1.4  -3.2  -1.7  -1.2  -2.1  -1.4  -1.4  -3.2  -1.3  -1.2  -1.4  -1.4  -1.2  -1.9  -1.2  -1.4  -2.4  -0.4  -2.4  -2.8  -2.4  -2.4  -2.4  -2.4  
oox.  .oxo  .ox.  .ox.  .ox.  .ox.  .ox.  .ox.  .ox.  .ox.  .ox.  oox.  .oxo  .ox.  .ox.  .ox.  .ox.  .ox.  .ox.  .ox.  oox.  .oxo  .ox.  .ox.  .ox.  .ox.  .ox.  .ox.  .ox.  .ox.  .ox.  oox.  .ox.  .ox.  .ox.  .ox

In [ ]:
history = [[[1,0,0,1],[0,0,1,1]],[[1,1,1,1],[0,0,0,0]]]
h2 = history[-1:]
h2.append([[1,1,1,1],[0,1,0,1]])
print(h2)
print(history)

[[[1, 1, 1, 1], [0, 0, 0, 0]], [[1, 1, 1, 1], [0, 1, 0, 1]]]
[[[1, 0, 0, 1], [0, 0, 1, 1]], [[1, 1, 1, 1], [0, 0, 0, 0]]]


In [ ]:
game3.replay(tree3.head.history)

0:

Score: B/1  W/0
- o x - 
- x - - 
- - - - 
- - - - 

1:

Score: B/1  W/0
- o x - 
- x - - 
o - - - 
- - - - 

2:

Score: B/1  W/0
x - x - 
- x - - 
o - - - 
- - - - 

3:

Score: B/1  W/0
x - x - 
o x - - 
o - - - 
- - - - 

4:

Score: B/1  W/0
x - x - 
o x - - 
o x - - 
- - - - 



In [ ]:
tmp_mat = val_matrix(tree3.head.board,0,tree3.head.history)
doMatrix = 1 / (tmp_mat + 9)
#doMatrix = PromAndRandom(tmp_mat)
print(doMatrix)
for i in range(4):
  for j in range(4):
    if doMatrix[i,j]:
      print(i,j)

[[ 0.1        -0.         -0.          0.11111111]
 [-0.         -0.          0.11111111  0.11111111]
 [-0.         -0.          0.11111111  0.11111111]
 [ 0.11111111  0.11111111  0.11111111  0.11111111]]
0 0
0 3
1 2
1 3
2 2
2 3
3 0
3 1
3 2
3 3


In [ ]:
val_matrix(tree3.head.board, 0, tree3.head.history)

array([[  1., -inf, -inf,   0.],
       [-inf, -inf,   0.,   0.],
       [-inf, -inf,   0.,   0.],
       [  0.,   0.,   0.,   0.]])

In [ ]:
val_matrix(game3.board, 0, game3.history)

array([[-inf,   0., -inf,   0.],
       [-inf, -inf,   0.,   0.],
       [-inf, -inf,   0.,   0.],
       [  0.,   0.,   0.,   0.]])

In [ ]:
treee = GameTree(game4)
treee.lookForward(3, 4)
treee.displayTree(2,0)

10
98
788
2468
6383
14807
32707
=0.0  
..x.  
.xo.  
.xo.  
.o..  

=0.0  =0.0  -0.7  -0.3  -0.4  =0.1  -0.4  -0.7  -0.8  -0.2  
x.x.  .xx.  ..xx  ..x.  ..x.  ..x.  ..x.  ..x.  ..x.  ..x.  
.xo.  .xo.  .xo.  xxo.  .xox  .xo.  .xo.  .xo.  .xo.  .xo.  
.xo.  .xo.  .xo.  .xo.  .xo.  xxo.  .xox  .xo.  .xo.  .xo.  
.o..  .o..  .o..  .o..  .o..  .o..  .o..  xo..  .ox.  .o.x  



In [ ]:
treee = GameTree(game4)
treee.lookForward(3,2)
treee.displayTree(2,0)

10
98
788
2429
6207
=0.0  
..x.  
.xo.  
.xo.  
.o..  

=0.0  =0.0  -0.0  =0.0  -0.4  =0.1  -0.5  -0.7  -0.4  -0.5  
x.x.  .xx.  ..xx  ..x.  ..x.  ..x.  ..x.  ..x.  ..x.  ..x.  
.xo.  .xo.  .xo.  xxo.  .xox  .xo.  .xo.  .xo.  .xo.  .xo.  
.xo.  .xo.  .xo.  .xo.  .xo.  xxo.  .xox  .xo.  .xo.  .xo.  
.o..  .o..  .o..  .o..  .o..  .o..  .o..  xo..  .ox.  .o.x  



In [ ]:
tree.head.proceed(0,3)
tree.head.children[1].last() == (3,2)

False

In [ ]:
b1 = np.matrix(f'''
1  0;
1  2
''')
b3 = np.full((3,3), np.NINF)
b3 [1,1] = 5
b2 = np.matrix('''
2  1;
2  1
''')
b1 == b2

matrix([[False, False],
        [False, False]])

In [ ]:
board = np.matrix('''
-1  0  0  1 -1;
1  1  0  1  1;
1  1  1  0  0;
0  0  1  0  1;
1  0  1  0  1
''')

def printb(board):
  for i in range(5):
    for j in range(5):
      if board[i,j] >= 0:
        if not board[i,j]:
          print('x', end = " ")
        else:
          print('o', end = " ")
      else:
        print('.', end = ' ')
    print()


for i in range(5):
  for j in range(5):
    if board[i, j] >= 0:
      rec, lib = cluster(board, i, j, {})
      print(int(lib), end = " ")
    else:
      print("-", end=' ')
  print()

print()

printb(board)

print()
a,b = outcome(board, 3, 3)

printb(b)
print(a)

- 1 1 1 - 
1 1 1 1 1 
1 1 1 0 0 
0 0 1 0 0 
0 0 1 0 0 

. x x o . 
o o x o o 
o o o x x 
x x o x o 
o x o x o 

. x x o . 
o o x o o 
o o o x x 
x x o x . 
o x o x . 
2


In [ ]:
_dic = {'a,b':5, 'b,c':6}
for i in _dic:
  tmp = i.split(',')
  print(tmp[0])

a
b


In [ ]:
_n = np.full((5,5), -1)
print(_n)
for i in  game1.history[-3:]:
  if np.array_equal(i, _n):
    print('Yes')
    print(np.matrix(i))

print(game1.history)

[[-1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1]]
[array([[-1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1]]), array([[-1,  0, -1, -1, -1],
       [-1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1]]), array([[-1,  0, -1,  1, -1],
       [-1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1]]), array([[-1,  0, -1,  1, -1],
       [-1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1],
       [ 0, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1]]), array([[-1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1]]), array([[-1,  0, -1, -1, -1],
       [-1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1]]), array([[-1,  0, -1,  1, -1